# Try to classify contigs as prokaryotic or eukaryotic just from the alignment

General idea: eukaryotic contigs, compared with prokaryotic contigs, should have relatively high amounts of positions with variants around 50% frequency (representing diploidy).

In [2]:
%run "Header.ipynb"

In [3]:
import math
import json
import pysam
import pysamstats

In [8]:
# No edges with length < this will be considered here
MIN_SEQ_LEN = 5000

# Minimum coverage for positions where we want to look for ~50% mutation rates.
# This is purposefully low so as to cast a wide net.
MIN_COV = 5

# Both of these are inclusive. We'll accept positions (with at least MIN_COV aligned (mis)matches)
# where the maximum-frequency alternate nucleotide's frequency (divided by total # of aligned (mis)matches)
# is within this inclusive range.
#
# ...So, for a position with exactly 5x (mis)match coverage, we'd accept a max-freq alt nucleotide freq of
# [2, 3] but not 1, 4, or 5.
MIN_DIPLOID_MAXALT_FREQ = 0.4
MAX_DIPLOID_MAXALT_FREQ = 0.6

In [5]:
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")

## Get a mapping of sequence (edge) name to sequence length

stolen from the diversity index notebook; should merge these into a single function

also the div index notebook version of this accidentally trumps the `seq2len` variable defined in the header notebook... should fix that, but not super impt

In [6]:
edge2len = {}
edgename = None
with open("../main-workflow/output/all_edges.fasta", "r") as fastafile:
    
    # Assumes that sequences are not split up over multiple lines (so a FASTA file with N sequences
    # should have only 2N lines, maybe 2N + 1 if there's an extra empty newline at the bottom of the file)
    for linenum, line in enumerate(fastafile):
    
        if line.startswith(">"):
            if linenum % 2 != 0:
                raise ValueError("something weird with > location in all_edges.fasta. Go yell at Marcus.")
            edgename = line.strip()[1:]
        else:
            if linenum % 2 != 1:
                raise ValueError("something weird with non > location in all_edges.fasta. Go yell at Marcus.")
            if edgename is None:
                raise ValueError("No sequence name set yet?")
            edge2len[edgename] = len(line.strip())
            edgename = None

print(f"edge2len contains {len(edge2len):,} entries. The length of edge_1 is {edge2len['edge_1']:,} bp.")

edge2len contains 78,793 entries. The length of edge_1 is 39,565 bp.


In [7]:
long_enough_seqs = []
for edge in edge2len:
    if edge2len[edge] >= MIN_SEQ_LEN:
        long_enough_seqs.append(edge)
        
pct = 100 * (len(long_enough_seqs) / len(edge2len))
print(f"{len(long_enough_seqs):,} / {len(edge2len):,} ({pct:.2f}%) edges have a length of at least {MIN_SEQ_LEN:,.0f} bp.")

43,562 / 78,793 (55.29%) edges have a length of at least 5,000 bp.


## Go through the alignment; count the number of positions with ~50% max-frequency alternate nucleotide mismatches

Parts of this (e.g. processing each record in the pileup) could be abstracted to code that's shared between here, the diversity index ntbk, and the bam2pileup script I wrote for the three "selected" MAGs.

In [ ]:
edge2num_mincov_positions = {}
edge2num_mincov_positions_with_at_least_one_mismatch = {}
edge2num_mincov_putatively_diploid_positions = {}
edge2meancov = {}

num_seqs = len(long_enough_seqs)

for si, seq in enumerate(long_enough_seqs, 1):
    pct = 100 * (si / num_seqs)
    print(f"On seq {seq} ({si:,} / {num_seqs:,}) ({pct:.2f}%)...", end=" ", flush=True)   
    print(f"Length = {edge2len[seq]:,} bp.", end=" ", flush=True)
    
    one_tenth_len = math.floor(edge2len[seq] / 10)
    
    num_mincov_positions = 0
    num_mincov_positions_with_at_least_one_mismatch = 0
    num_mincov_putatively_diploid_positions = 0
    
    total_cov = 0
    
    # Go through each position in the sequence's pileup (even including relatively low-coverage positions).
    # See bam-to-jsons.py for a description of why these params are useful.
    # Also, note that pysam coords are 0-indexed!
    for ri, rec in enumerate(
        pysamstats.stat_variation(
            bf, chrom=seq, fafile="../main-workflow/output/all_edges.fasta", start=0, end=edge2len[seq],
            truncate=True, pad=True, max_depth=1e6
        ),
        1
    ):
        # very simple progress bar for my own sake. might miss a few dots here and there
        if ri % one_tenth_len == 0:
            print("*", end="", flush=True)
            
        matches = rec["matches"]
        mismatches = rec["mismatches"]
        # NOTE: As with "coverage" in the coverage plots, this doesn't take into account deletions at a
        # given position. That info is available through pysamstats so we could use it here if desired.
        cov = matches + mismatches
        
        total_cov += cov
        
        #print(f"Pos {ri}... Matches: {matches} / Mismatches: {mismatches} / cov: {cov}")
        
        if cov > MIN_COV:
            num_mincov_positions += 1
            # Rather than looking at (mismatches / cov), we instead look at each alternate non-matching nucleotide
            # individually.
            nonmatches = set("ACGT") - set(rec["ref"])
            max_alt_nt_freq = max(rec[alt_nt] for alt_nt in nonmatches)
            max_alt_nt_relfreq = max_alt_nt_freq / cov
            
            # Keep track of # (min-cov) positions that have at least one mismatch; could be useful to know
            if max_alt_nt_freq >= 1:
                num_mincov_positions_with_at_least_one_mismatch += 1
                #print("\tHas at least 1 mismatch!")
                
            # The meat of this: see if this position "counts" as putatively diploid due to having a ~50%
            # mutation rate
            if max_alt_nt_relfreq >= MIN_DIPLOID_MAXALT_FREQ and max_alt_nt_relfreq <= MAX_DIPLOID_MAXALT_FREQ:
                num_mincov_putatively_diploid_positions += 1
                #print(f"\tPutatively diploid! rel freq = {max_alt_nt_relfreq}")
    
    assert ri == edge2len[seq]
    meancov = total_cov / ri
    
    # Save info for each edge
    edge2num_mincov_positions[seq] = num_mincov_positions
    edge2num_mincov_positions_with_at_least_one_mismatch[seq] = num_mincov_positions_with_at_least_one_mismatch
    edge2num_mincov_putatively_diploid_positions[seq] = num_mincov_putatively_diploid_positions
    edge2meancov[seq] = meancov
    
    # Move print info to a new line
    print(f"\n\t{num_mincov_positions:,} pos w/ cov >= {MIN_COV}x;", end=" ")
    print(f"{num_mincov_positions_with_at_least_one_mismatch:,} MCP w/ >= 1 mismatch;", end=" ")
    print(f"{num_mincov_putatively_diploid_positions:,} 'diploid' MCP;", end=" ")
    print(f"mean cov = {meancov:,.2f}x")

On seq edge_1 (1 / 43,562) (0.00%)... Length = 39,565 bp. **********
	39,558 pos w/ cov >= 5x; 6,310 MCP w/ >= 1 mismatch; 13 'diploid' MCP; mean cov = 460.40x
On seq edge_3 (2 / 43,562) (0.00%)... Length = 38,161 bp. **********
	36,963 pos w/ cov >= 5x; 3,805 MCP w/ >= 1 mismatch; 11 'diploid' MCP; mean cov = 258.76x
On seq edge_5 (3 / 43,562) (0.01%)... Length = 32,079 bp. **********
	5,867 pos w/ cov >= 5x; 104 MCP w/ >= 1 mismatch; 0 'diploid' MCP; mean cov = 3.36x
On seq edge_6 (4 / 43,562) (0.01%)... Length = 6,738 bp. **********
	0 pos w/ cov >= 5x; 0 MCP w/ >= 1 mismatch; 0 'diploid' MCP; mean cov = 1.40x
On seq edge_8 (5 / 43,562) (0.01%)... Length = 1,710,962 bp. *********

### Save info to a file

Since this can take a few hours to compute, this helps if we have to restart the notebook halfway through / etc.

In [ ]:
with open("misc-output/prok-euk-info.txt", "w") as pef:
    pef.write(json.dumps(edge2num_mincov_positions))
    pef.write(json.dumps(edge2num_mincov_positions_with_at_least_one_mismatch))
    pef.write(json.dumps(edge2num_mincov_putatively_diploid_positions))
    pef.write(json.dumps(edge2meancov))